OUTLINE

MODULE 1: Data preprocessing and loading

(a) Obtaining and cleaning the datasets

(b) Twitter specific text pre-processing

(c) Word / sentence vectorisation (as first data input)

Michael: BERT and FastText

(d) Implementing a dictionary approach potentially based on Hatebase.org (as second data input)
(e) Splitting data into test, validation and testing set
(f) Specifying and implementing the data loader
(g) Testing and iterating the module
(h) Creatingmodule-specificvisualisationsforthefinal paper

MODULE 2: Model architecture and training

(a) Choosing width and depth of the model
(b) Choosingoptimizeraswellasactivationandloss functions
(c) Choosing stopping rule, regularisation, dropout, learning rates etc.
(d) Potentially performing a hyperparameter grid search
(e) Running and tracking the training
(f) Training and tuning the model based on the re- sults of the validation set
(g) Creatingmodule-specificvisualisationsforthefi- nal paper

In [12]:
import pandas as pd

In [13]:
# (a) Obtaining and cleaning the datasets
url_davidson = "https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/data/twitter%20data/davidson%20et%20al./labeled_data.csv"
url_founta = "https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/data/twitter%20data/founta%20et%20al./hatespeech_text_label_vote.csv"


data_davidson = pd.read_csv(url_davidson, names=["count", "hate_speech", "offensive_language", "neither", "label", "tweet"], header=1)
data_founta = pd.read_csv(url_founta, sep='\t', names=["tweet", "label_text", "count"])

data_founta = data_founta[~data_founta.label_text.str.contains("spam")]
data_founta['label'] = data_founta.label_text.replace({'hateful': '0', 'abusive': '1', 'normal': '2'}).astype('int')
data_founta.label.hist()

# concatinating and shuffling


HTTPError: HTTP Error 404: Not Found

In [ ]:
data_davidson.label.hist()

In [ ]:
import pandas as pd
df = pd.read_csv("/Users/mxm/Google Drive/Masterstudium/Inhalte/4th Semester/NLP/nlp-project/coding/code/exchange_base/data.csv")
df.label.hist()

In [ ]:
# trying to re-import hatebase, somehow doesn't work

'''
import json 
import requests
import pandas as pd
from hatebase import HatebaseAPI

key = open('C:/Users/Niko/Documents/NJS Cloud/1 Uni (non-Google)/0 Hertie/2 Spring 2020/NLP/Hatebase_Key', "r").read() # insert filepath where the API key is stored
filepath = "C:/Users/Niko/Documents/NJS Cloud/1 Uni (non-Google)/0 Hertie/2 Spring 2020/NLP/full_dictionary.csv" # insert filepath where the final csv file should be stored 

hatebase = HatebaseAPI({"key": key})
filters = {"language": "eng"}
format = "json"
# initialize list for all vocabulary entry dictionaries
eng_vocab = []
response = hatebase.getVocabulary(filters=filters, format=format)
pages = response["number_of_pages"]
# fill the vocabulary list with all entries of all pages
# this might take some time...
for page in range(1, pages+1):
    filters["page"] = str(page) 
    response = hatebase.getVocabulary(filters=filters, format=format)
    eng_vocab.append(response["result"])

data = []
for r in eng_vocab:
    data.append(r["data"])
#print len(data)
listofHatewords = []

#print len(data)
for z in data:
    for a, v in z.iteritems():
        for b in v:
            listofHatewords.append(b["vocabulary"])
# print listofHatewords
listofHatewords = list(OrderedDict.fromkeys(listofHatewords))
print(len(listofHatewords))
'''


In [27]:
# Loading packages
#import csv
#import sys
#import nltk
import pandas as pd
import torch
import spacy
import dframcy
import pandarallel

#def dict_approach(data = None, lemmatization = True)
    """This function matches the terms in the Hatebase.org dictionary with the tweets in our dataset.
    First, both datasets are lemmatized, then the function counts the number of hatebase.org terms appearing in each tweet and adds the count as well as the average offensiveness (as defined by hatebase.org methodology). These two outputs are then added to the original dataset and further transformed into a tensor for further analysis.
    The input dataframe (tweets) can be specified with the variable "data". If nothing is specified, the function will get the data from our GitHub repository."""

# Loading the data

# loading Hatebase dictionary
hatebase_url = "https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/data/dictionary/hatebase/full_dictionary.csv"
hatebase_dic = pd.read_csv(hatebase_url, index_col = 'vocabulary_id')

# loading the tweets
data_url = "https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/code/exchange_base/data.csv"

# reading the data in case it is not here already (since it will later all be applied in one pipeline)
if data is None:
    data = pd.read_csv(data_url, index_col = 'id')
    print('INFO: Reading data anew from GitHub since no input was provided')


In [24]:
# Lemmatizing both the dictionary and the tweets
#if lemmatization:
# loading the spacy model
nlp = spacy.load('en_core_web_sm')
# switching off irrelevant spacy functions
nlp.disable_pipes('tagger', 'ner')

# lemmatizing the dictionary
lemma = []
for term in nlp.pipe(hatebase_dic['term'].astype('unicode').values, batch_size=50, n_threads=3):
    if term.is_parsed:
        lemma.append([n.lemma_ for n in term])
    else:
        lemma.append(None)

# storing the lemmas in a new column 
hatebase_dic['term_lemma'] = lemma

#making lemmas lowercase
hatebase_dic['term_lemma'] = hatebase_dic['term_lemma'].map(lambda lemmas: [x.lower() for x in lemmas])
    
#joining list of lemmas into one string agein, so it becomes matchable by the searchfunction
hatebase_dic['term_lemma'] = hatebase_dic['term_lemma'].map(lambda lemmas: " ".join(lemmas))

# lemmatizing the tweets
lemma = []
for tweet in nlp.pipe(data['tweet'].astype('unicode').values, batch_size=50, n_threads=3):
    if tweet.is_parsed:
        lemma.append([n.lemma_ for n in tweet])
    else:
        lemma.append(None)

# storing the lemmas in a new column 
data['tweet_lemma'] = lemma

#making lemmas lowercase
data['tweet_lemma'] = data['tweet_lemma'].map(lambda lemmas: [x.lower() for x in lemmas])
    
#joining list of lemmas into one string agein, so it becomes matchable by the searchfunction
data['tweet_lemma'] = data['tweet_lemma'].map(lambda lemmas: " ".join(lemmas))

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [22]:
# Searching for HateBase words in tweets

pandarallel.initialize() 

# defining the search function
def hatesearch(data):
    frequency = 0
    hatefulness = 0
    for term_lemma in hatebase_dic:
        if term_lemma in data['tweet_lemma']:
            frequency += 1
            hatefulness = hatebase_dic[hatebase_dic['term_lemma'] == term_lemma].average_offensiveness
    data['Hatefreq'] = frequency
    data['Hatefulness'] = hatefulness
    return data

# running the search function
data = data.parallel_apply(hatesearch, axis = 1)

#data = data.apply(lambda x : hatesearch, axis=1)

#print(data)

# Exporting to browse through

#data.to_csv('C:/Users/Niko/Documents/newdata.csv')

AttributeError: module 'pandarallel' has no attribute 'initialize'

In [21]:
# Building a tensor out of the additional columns

HateFrequency = torch.tensor(data['Hatefreq'])
torch.save(HateFrequency, 'https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/code/exchange_base/HateFreq_Tensor.pt')

HateIntensity = torch.tensor(data['Hatefulness'])
torch.save(HateFrequency, 'https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/code/exchange_base/HateIntens_Tensor.pt')

KeyError: 'Hatefreq'

In [18]:

# loading packages
import pandas as pd
import numpy as np
import transformers
import torch
import tqdm
import spacy
from pandarallel import pandarallel

# Loading the dictionary

hatebase_path = '../exchange_base/full_dictionary.csv'
hatebase_dic = pd.read_csv(hatebase_path, index_col = 'vocabulary_id')

# loading the spacy model
nlp = spacy.load('en_core_web_sm')
# switching off irrelevant spacy functions
nlp.disable_pipes('tagger', 'ner')

# lemmatizing the dictionary
lemma = []
for term in nlp.pipe(hatebase_dic['term'].astype('unicode').values, batch_size=50, n_threads=3):
    if term.is_parsed:
        lemma.append([n.lemma_ for n in term])
    else:
        lemma.append(None)

# storing the lemmas in a new column 
hatebase_dic['term_lemma'] = lemma

#making lemmas lowercase
hatebase_dic['term_lemma'] = hatebase_dic['term_lemma'].map(lambda lemmas: [x.lower() for x in lemmas])
    
#joining list of lemmas into one string again, so it becomes matchable by the searchfunction
hatebase_dic['term_lemma'] = hatebase_dic['term_lemma'].map(lambda lemmas: " ".join(lemmas))

hatebase_dic = hatebase_dic.drop_duplicates(subset=['term_lemma'])

# Loading the tensors

train_vectorized = torch.load('../exchange_base/train_vectorized_1d.pt')
print("List of vectorized tweets: ",train_vectorized)
print("One vectorized tweet: ",train_vectorized[0])
print("One word: ", train_vectorized[0][0])

# Loading BERT

pretrainedModel="bert-base-uncased"
tokenizer = transformers.BertTokenizer.from_pretrained(pretrainedModel)

# Decoding

#decodedtweets=pd.DataFrame([], index=['text'], columns=['tweetwords'])

decoded_tweet=[]
decoded_word=[]
frequency = 0
hatefulness_sum = 0
a = torch.Tensor=([])
b = torch.Tensor=([])
word = tweet[i]


for i,tweet in enumerate(train_vectorized):
    decoded_tweet = tokenizer.decode(tweet, skip_special_tokens=False, clean_up_tokenization_spaces = False)
    print("One decoded Tweet: ", decoded_tweet)
    for n,word in enumerate(tweet):
        decoded_word = tokenizer.decode(word.item(),skip_special_tokens=False, clean_up_tokenization_spaces = False)
    #df = pd.DataFrame([decoded_tweet], index=["text"], columns=['tweetwords'])
    #print ("One decoded tweet in dataframe: ",df)
    #df = df.tweetwords.apply(lambda x: pd.Series(str(x).split(" ")))
    print ("One decoded tweet in split words: ",df)
    def hatesearch(column):
        frequency = 0
        hatefulness_sum = 0
        for hateterm in hatebase_dic['term_lemma']:
            if hateterm in decoded_word:
                frequency += 1
                row_hatebase_dict_term_lemma = hatebase_dic.loc[hatebase_dic['term_lemma'] == hateterm, 'average_offensiveness'].iloc[0]
                if np.isnan(row_hatebase_dict_term_lemma):
                    hatefulness_term = 77.27734806629834
                else: 
                    hatefulness_term = row_hatebase_dict_term_lemma
                if hatebase_dic.loc[hatebase_dic['term_lemma'] == hateterm, 'is_unambiguous'].iloc[0] == False:
                    hatefulness_term_weighted = hatefulness_term
                else:
                    hatefulness_term_weighted = hatefulness_term*2
                hatefulness_sum += hatefulness_term_weighted
        column['Hatefreq'] = frequency
        if frequency != 0:
            column['Hatefulness'] = (hatefulness_sum / frequency)
        else:
            column['Hatefulness'] = 0
        return column
        #print("Decoded Tweet", decoded_tweet, "Hatefreq", Hatefreq, "Hatefulness", Hatefulness)
        df = df.apply(hatesearch, axis = 0)
        print(df)

        a = torch.tensor(df['Hatefreq'])
        b = torch.tensor(df['Hatefulness'])
        c = train_vectorized[i]
    if i == 1:
        break

merged_tensor = torch.stack([a,b,c], dim = 3)

print(merged_tensor)
## TODO


#'''ahead with one tweet: works!

tweet1 = train_vectorized[0]
print(type(tweet1))
print(tweet1)
'''
#tweets = tokenizer.decode(train_vector1zed, skip_special_tokens=True, clean_tweet1_up_tokenization_spaces = True) ### does not work!

df = pd.DataFrame([tweet1], index=["text"], columns=['tweetwords'])
print (df)

df = df.tweetwords.apply(lambda x: pd.Series(str(x).split(" ")))
print(df)

def hatesearch(column):
    frequency = 0
    hatefulness_sum = 0
    for hateterm in hatebase_dic['term_lemma']:
        if hateterm in df:
            frequency += 1
            row_hatebase_dict_term_lemma = hatebase_dic.loc[hatebase_dic['term_lemma'] == hateterm, 'average_offensiveness'].iloc[0]
            if np.isnan(row_hatebase_dict_term_lemma):
                hatefulness_term = 77.27734806629834
            else: 
                hatefulness_term = row_hatebase_dict_term_lemma
            if hatebase_dic.loc[hatebase_dic['term_lemma'] == hateterm, 'is_unambiguous'].iloc[0] == False:
                hatefulness_term_weighted = hatefulness_term
            else:
                hatefulness_term_weighted = hatefulness_term*2
            hatefulness_sum += hatefulness_term_weighted
    column['Hatefreq'] = frequency
    if frequency != 0:
        column['Hatefulness'] = (hatefulness_sum / frequency)
    else:
        column['Hatefulness'] = 0
    return column

#pandarallel.initialize() 
#df = df.parallel_apply(hatesearch, axis = 0)

## somehow had a problem with that

df = df.apply(hatesearch, axis = 0)

print(df)

# Building a tensors from the dataframe

HateFrequency = torch.tensor(data['Hatefreq'].values.astype("float16"))
HateIntensity = torch.tensor(data['Hatefulness'].values.astype("float16"))

return HateFrequency, HateIntensity

# output_file_name = "exchange_base/hatefreq.pt"
# 2. use exchange_base files

path = "coding/code/exchange_base/"

output_file_name_hatefreq_alt = path +  "hatefreq_alt.pt"
output_file_name_hateint_alt = path + "hateint_alt.pt"

# Saving the frequency tensor
torch.save(HateFrequency, output_file_name_hatefreq_alt)
# Saving the intesity tensor
torch.save(HateIntensity, output_file_name_hateint_alt)
'''

List of vectorized tweets:  tensor([[  101,  2017,  2228,  ...,     0,     0,     0],
        [  101,  2882,  2003,  ...,     0,     0,     0],
        [  101,  6842,  2380,  ...,     0,     0,     0],
        ...,
        [  101,  2217,  7743,  ...,     0,     0,     0],
        [  101, 19522,  2632,  ...,     0,     0,     0],
        [  101,  2179,  1037,  ...,     0,     0,     0]], dtype=torch.int32)
One vectorized tweet:  tensor([  101,  2017,  2228,  1996,  2293,  6771,  2005,  7055,  2097,  2058,
        15637,  2008,  2002,  1005,  1055,  3920,  1998,  2062,  3144,  2054,
         2515,  2008,  2812,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0, 

NameError: name 'i' is not defined

In [19]:
# loading packages
import pandas as pd
import numpy as np
import transformers
import torch
import tqdm
import spacy
from pandarallel import pandarallel

# Loading the dictionary

hatebase_path = '../exchange_base/full_dictionary.csv'
hatebase_dic = pd.read_csv(hatebase_path, index_col = 'vocabulary_id')

# loading the spacy model
nlp = spacy.load('en_core_web_sm')
# switching off irrelevant spacy functions
nlp.disable_pipes('tagger', 'ner')

# lemmatizing the dictionary
lemma = []
for term in nlp.pipe(hatebase_dic['term'].astype('unicode').values, batch_size=50, n_threads=3):
    if term.is_parsed:
        lemma.append([n.lemma_ for n in term])
    else:
        lemma.append(None)

# storing the lemmas in a new column 
hatebase_dic['term_lemma'] = lemma

#making lemmas lowercase
hatebase_dic['term_lemma'] = hatebase_dic['term_lemma'].map(lambda lemmas: [x.lower() for x in lemmas])
    
#joining list of lemmas into one string again, so it becomes matchable by the searchfunction
hatebase_dic['term_lemma'] = hatebase_dic['term_lemma'].map(lambda lemmas: " ".join(lemmas))

hatebase_dic = hatebase_dic.drop_duplicates(subset=['term_lemma'])


# Loading the data
data_path = "../exchange_base/train_set.csv"

# reading the data in case it is not here already (since it will later all be applied in one pipeline)

df = pd.read_csv(data_path, index_col = 'id')
print(df.head)

print((df['tweet'][0]))
# Splitting the tweets into individual strings
'''
print(type(df['tweet'][0]))
tweet1_split = (df['tweet'][0]).split()


print(tweet1_split)
'''


# tweetstrings = df['tweet'].apply(tokenizer.tokenize, axis = 0)

'''
VectorofTokenizedTweets
for tweet in data['tweet']:
    NewTweet = []
    NewTweet = tokenizer.tokenize(tweet)
VectorofTokenizedTweets.append(NewTweet)
'''
#hatebase_dic.to_csv('../exchange_base/Lemmatized_HateDic_with duplicates.csv')


<bound method NDFrame.head of        label  count                                              tweet
id                                                                    
9541       2      5  you think the love affair for butler will over...
66467      2      4   grand is just about topped off soon hundreds ...
12499      2      4  hudson park collection 800tc king sham cotton ...
58552      0      3  my mom always complaining how you always mad f...
72609      2      3   covington reads responses amp mocks fbi andy ...
...      ...    ...                                                ...
62097      2      5  so tel me how 'wonderful' capitalism is it has...
54140      2      4        it's see what kind of season eects from the
57935      1      3  side bitches are basically new relationships s...
58697      0      4  dani alves pulled off that hairstyle neymar ju...
14102      2      3  found a transponder snail a tearful farewell t...

[77522 rows x 3 columns]>
birthday surprises o

"\nVectorofTokenizedTweets\nfor tweet in data['tweet']:\n    NewTweet = []\n    NewTweet = tokenizer.tokenize(tweet)\nVectorofTokenizedTweets.append(NewTweet)\n"

In [2]:
# Splitting the tweets

tweet_i = []
for tweet in df['tweet']:
    tweet_split = str(tweet).split()
    tweet_i.append(tweet_split)
df['tweet_split'] = tweet_i
print(df.head)

<bound method NDFrame.head of        label  count                                              tweet  \
id                                                                       
9541       2      5  you think the love affair for butler will over...   
66467      2      4   grand is just about topped off soon hundreds ...   
12499      2      4  hudson park collection 800tc king sham cotton ...   
58552      0      3  my mom always complaining how you always mad f...   
72609      2      3   covington reads responses amp mocks fbi andy ...   
...      ...    ...                                                ...   
62097      2      5  so tel me how 'wonderful' capitalism is it has...   
54140      2      4        it's see what kind of season eects from the   
57935      1      3  side bitches are basically new relationships s...   
58697      0      4  dani alves pulled off that hairstyle neymar ju...   
14102      2      3  found a transponder snail a tearful farewell t...   

       

In [16]:
# BERT vectorizer

pretrainedModel="bert-base-uncased"
tokenizer = transformers.BertTokenizer.from_pretrained(pretrainedModel)
'''
TweetVector = []
for word in df['tweet_split'][0]:
    vectorized_tweet = tokenizer.encode(word)
    TweetVector.append(vectorized_tweet)
print(TweetVector)
TweetTensor = torch.tensor(TweetVector)
#print(TweetTensor)
'''
tweet_example = df['tweet_split'][0]
print(tweet_example)
vectorized_tweet = tokenizer.encode(tweet_example)
print(vectorized_tweet)
TweetTensor = torch.tensor(vectorized_tweet)

print(TweetTensor)

['birthday', 'surprises', 'on', 'holiday', 'in', 'cala', 'bona', 'marjorca', 'gran', 'sol', 'hotel', 'cala', 'bona']
[101, 5798, 20096, 2006, 6209, 1999, 100, 100, 100, 12604, 14017, 3309, 100, 100, 102]
tensor([  101,  5798, 20096,  2006,  6209,  1999,   100,   100,   100, 12604,
        14017,  3309,   100,   100,   102])


In [17]:
# Dictionary on a vector

samplevector = (["your", "wagon", "fish", "chief", "you", "retarded", "faggot"], ["you", "jewtarded", "fucking", "faggot", "you", "ass", "chief"])
HateVector = []

for tweet in samplevector:
    HateList=[]
    for word in tweet:
        frequency = 0
        hatefulness_term_weighted = 0
        offensiveness_value = 0
        Hatefulness = 0
        print(word)
        for hateterm in hatebase_dic['term_lemma']:
            frequency = 0
            hatefulness_term_weighted = 0
            offensiveness_value = 0
            if hateterm == word:
                frequency += 1
                offensiveness_value = hatebase_dic.loc[hatebase_dic['term_lemma'] == hateterm, 'average_offensiveness'].iloc[0]
                print("Single offensiveness value is", offensiveness_value)
                #print(frequency)
                if np.isnan(offensiveness_value):
                    offensiveness_value = 77.27734806629834
                else:
                    offensiveness_value = offensiveness_value
                if hatebase_dic.loc[hatebase_dic['term_lemma'] == hateterm, 'is_unambiguous'].iloc[0] == False:
                    hatefulness_term_weighted = offensiveness_value
                else:
                    hatefulness_term_weighted = offensiveness_value*2
                print("Weighted hatefulness value is", hatefulness_term_weighted)
                #print(frequency)
                if frequency != 0:
                    Hatefulness = hatefulness_term_weighted
                else:
                    Hatefulness = 0
        HateList.append(Hatefulness)
    HateVector.append(HateList)
print(HateVector)

HateTensor = torch.tensor(HateVector)
print(HateTensor)

your
wagon
fish
chief
Single offensiveness value is 85.0
Weighted hatefulness value is 85.0
you
retarded
faggot
Single offensiveness value is 92.0
Weighted hatefulness value is 184.0
you
jewtarded
Single offensiveness value is 98.0
Weighted hatefulness value is 196.0
fucking
faggot
Single offensiveness value is 92.0
Weighted hatefulness value is 184.0
you
ass
chief
Single offensiveness value is 85.0
Weighted hatefulness value is 85.0
[[0, 0, 0, 85.0, 0, 0, 184.0], [0, 196.0, 0, 184.0, 0, 0, 85.0]]
tensor([[  0.,   0.,   0.,  85.,   0.,   0., 184.],
        [  0., 196.,   0., 184.,   0.,   0.,  85.]])


In [20]:
# Dictionary on a vector

samplevector = (["your", "wagon", "fish", "chief", "you", "retarded", "fag"], ["you", "jewtarded", "fucking", "fag", "you", "ass", "chief"])
HateVector = []

for sentence in samplevector:
    HateList=[]
    for word in sentence:
        frequency = 0
        hatefulness_term_weighted = 0
        offensiveness_value = 0
        Hatefulness = 0
        print(word)
        for hateterm in hatebase_dic['term_lemma']:
            frequency = 0
            hatefulness_term_weighted = 0
            offensiveness_value = 0
            if hateterm == word:
                frequency += 1
                offensiveness_value = hatebase_dic.loc[hatebase_dic['term_lemma'] == hateterm, 'average_offensiveness'].iloc[0]
                print("Single offensiveness value is", offensiveness_value)
                #print(frequency)
                if np.isnan(offensiveness_value):
                    offensiveness_value = 77.27734806629834
                else:
                    offensiveness_value = offensiveness_value
                if hatebase_dic.loc[hatebase_dic['term_lemma'] == hateterm, 'is_unambiguous'].iloc[0] == False:
                    hatefulness_term_weighted = offensiveness_value
                else:
                    hatefulness_term_weighted = offensiveness_value*2
                print("Weighted hatefulness value is", hatefulness_term_weighted)
                #print(frequency)
                if frequency != 0:
                    Hatefulness = hatefulness_term_weighted
                else:
                    Hatefulness = 0
        HateList.append(Hatefulness)
    HateVector.append(HateList)
print(HateVector)

your
wagon
fish
chief
Single offensiveness value is 85.0
Weighted hatefulness value is 85.0
you
retarded
fag
Single offensiveness value is nan
Weighted hatefulness value is 77.27734806629834
you
jewtarded
Single offensiveness value is 98.0
Weighted hatefulness value is 196.0
fucking
fag
Single offensiveness value is nan
Weighted hatefulness value is 77.27734806629834
you
ass
chief
Single offensiveness value is 85.0
Weighted hatefulness value is 85.0
[[0, 0, 0, 85.0, 0, 0, 77.27734806629834], [0, 196.0, 0, 77.27734806629834, 0, 0, 85.0]]


In [12]:
# Works for a sentence - for alternative way, see sampleword example below 

samplesentence = (["your"], ["ass"], ["fucking"], ["chief"], ["you"], ["jewtarded"], ["fag"])
HateList = []

for word in df['tweet_split'][0]:
    frequency = 0
    hatefulness_term_weighted = 0
    offensiveness_value = 0
    Hatefulness = 0
    print(word)
    for hateterm in hatebase_dic['term_lemma']:
        frequency = 0
        hatefulness_term_weighted = 0
        offensiveness_value = 0
        if hateterm == word:
            frequency += 1
            offensiveness_value = hatebase_dic.loc[hatebase_dic['term_lemma'] == hateterm, 'average_offensiveness'].iloc[0]
            print("Single offensiveness value is", offensiveness_value)
            #print(frequency)
            if np.isnan(offensiveness_value):
                offensiveness_value = 77.27734806629834
            else:
                offensiveness_value = offensiveness_value
            if hatebase_dic.loc[hatebase_dic['term_lemma'] == hateterm, 'is_unambiguous'].iloc[0] == False:
                hatefulness_term_weighted = offensiveness_value
            else:
                hatefulness_term_weighted = offensiveness_value*2
            print("Weighted hatefulness value is", hatefulness_term_weighted)
            #print(frequency)
            if frequency != 0:
                Hatefulness = hatefulness_term_weighted
            else:
                Hatefulness = 0
    HateList.append(Hatefulness)
print(HateList)

HateTensor = torch.tensor(HateList)
print(HateTensor)

matrix = torch.stack ([TweetTensor, HateTensor])
print(matrix)

birthday
surprises
on
holiday
in
cala
bona
marjorca
gran
sol
hotel
cala
bona
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 15 and 13 in dimension 1 at C:\Users\builder\AppData\Local\Temp\pip-req-build-9msmi1s9\aten\src\TH/generic/THTensor.cpp:689

In [22]:
data = ("your wagon fish chief you retarded faggot")

tweet_split = str(data).split()

# Searching through every word in the dictionary for every word in the tweet
HateVector = []

for word in tweet_split:
    frequency = 0
    hatefulness_term_weighted = 0
    offensiveness_value = 0
    Hatefulness = 0
    print(word)
    for hateterm in hatebase_dic['term']:
        frequency = 0
        hatefulness_term_weighted = 0
        offensiveness_value = 0
        if hateterm == word:
            frequency += 1
            offensiveness_value = hatebase_dic.loc[hatebase_dic['term'] == hateterm, 'average_offensiveness'].iloc[0]
            print("Single offensiveness value is", offensiveness_value)
            #print(frequency)
            if np.isnan(offensiveness_value):
                offensiveness_value = 77.27734806629834
            else:
                offensiveness_value = offensiveness_value
            if hatebase_dic.loc[hatebase_dic['term'] == hateterm, 'is_unambiguous'].iloc[0] == False:
                hatefulness_term_weighted = offensiveness_value
            else:
                hatefulness_term_weighted = offensiveness_value*2
            print("Weighted hatefulness value is", hatefulness_term_weighted)
            #print(frequency)
            if frequency != 0:
                Hatefulness = hatefulness_term_weighted
            else:
                Hatefulness = 0
    HateVector.append(Hatefulness)
print(HateVector)

# Building a tensor from the vector created
HateTensor = torch.tensor(HateVector)

your
wagon
fish
chief
Single offensiveness value is 85.0
Weighted hatefulness value is 85.0
you
retarded
Single offensiveness value is 64.0
Weighted hatefulness value is 128.0
faggot
[0, 0, 0, 85.0, 0, 128.0, 0]


In [17]:
# Works for one word
sampleword = ("chief")
'''
### Alternative: Look for sampleword in hatebase_dic
# Problem here: small words like "is" or "a" cause problems with the str.match function
for hateterm in sampleword:
    frequency = 0
    hatefulness_term_weighted = 0
    offensiveness_value = 0
    if hatebase_dic['term_lemma'].str.match(hateterm).any()
'''

for hateterm in hatebase_dic['term_lemma']:
    frequency = 0
    hatefulness_term_weighted = 0
    offensiveness_value = 0
    if hateterm == sampleword:
        frequency += 1
        offensiveness_value = hatebase_dic.loc[hatebase_dic['term_lemma'] == hateterm, 'average_offensiveness'].iloc[0]
        print(offensiveness_value)
        print(frequency)
        if np.isnan(offensiveness_value):
            offensiveness_value = 77.27734806629834
        else:
            offensiveness_value = offensiveness_value
        if hatebase_dic.loc[hatebase_dic['term_lemma'] == hateterm, 'is_unambiguous'].iloc[0] == False:
            hatefulness_term_weighted = offensiveness_value
        else:
            hatefulness_term_weighted = offensiveness_value*2
        print(hatefulness_term_weighted)
        print(frequency)
        if frequency != 0:
            Hatefulness = hatefulness_term_weighted
        else:
            Hatefulness = 0
print(Hatefulness)
print(sampleword)
print(hatebase_dic['term_lemma'].head)

85.0
1
85.0
1
85.0
chief
<bound method NDFrame.head of vocabulary_id
ayhoGnAxg           girl
oLVQyHuFA         whitie
WXianaa2P     fish wagon
CPZaHMujK          skank
w9CQ9KeW4    gun burglar
                ...     
yaA9bkM3c      celestial
DGzCe8c78        charlie
t9yPVBHhT             ho
WBwg3wFQC          chief
R6bhHFcHy         skinny
Name: term_lemma, Length: 1276, dtype: object>
